# L1 与 L2正则化
【参考】
- [github - spark-ml-source-analysis/最优化算法/L-BFGS](https://github.com/endymecy/spark-ml-source-analysis/blob/master/%E6%9C%80%E4%BC%98%E5%8C%96%E7%AE%97%E6%B3%95/L-BFGS/lbfgs.md)

在机器学习算法中，使用损失函数作为最小化误差，而最小化误差是为了让我们的模型拟合我们的训练数据，此时， 若参数过分拟合我们的训练数据就会有过拟合的问题。正则化参数的目的就是为了防止我们的模型过分拟合训练数据。此时，我们会在损失项之后加上正则化项以约束模型中的参数
$$
\large{
\begin{split}
J(x) = l(x) + r(x)
\end{split}
}
$$

公式右边的第一项 $l(x)$ 是损失函数，用来衡量当训练出现**偏差**时的损失，可以是任意可微凸函数（如果是非凸函数该算法只保证找到局部最优解）。 第二项 $r(x) $是正则化项。用来对模型空间进行限制，从而得到一个更“简单”的模型。

根据对模型参数所服从的概率分布的假设的不同，常用的正则化一般有L2正则化（模型参数服从 `Gaussian`分布）、L1正则化（模型参数服从`Laplace`分布）以及它们的组合形式。

`L1`(曼哈顿距离)正则化的形式如下
$$
\large{
\begin{split}
J(x) &= l(x) + C ||x||_{1} \\
dist_{man}(\boldsymbol{x}_i, \boldsymbol{x}_j) &= \Arrowvert \boldsymbol{x}_i - \boldsymbol{x}_j \Arrowvert_{1} = \sum_{u=1}^{n}\lvert x_{iu} - x_{ju}\rvert
\end{split}
}
$$

`L2`(欧式距离)正则化的形式如下：
$$
\large{
\begin{split}
J(x) &= l(x) + C ||x||_{2} \\
dist_{ed}(\boldsymbol{x}_i, \boldsymbol{x}_j) &= \Arrowvert \boldsymbol{x}_i - \boldsymbol{x}_j \Arrowvert_{2} = \sqrt{\sum_{u=1}^{n}\lvert x_{iu} - x_{ju}\rvert^{2}}
\end{split}
}
$$

## 正则化的意义

`L1` 正则化和 `L2` 正则化之间的一个最大区别在于前者可以产生稀疏解，这使它同时具有了特征选择的能力，此外，稀疏的特征权重更具有解释意义。如下图：
![image](https://ws3.sinaimg.cn/large/69d4185bly1fwv0qfuoy6j20em098wf3.jpg)

当模型中只有两个参数，即$w_1$和$w_2$时，`L2` 正则的约束空间是一个圆，而 `L1` 正则的约束空间为一个正方形，这样，基于`L1`正则的约束会产生稀疏解，即图中某一维 ($w_2$) 为0，值交点出现在某个轴上。 而 `L2` 正则只是将参数约束在接近 0 的很小的区间里，而不会正好为0(不排除有0的情况)，即交点出现在某个象限内。对于L1正则产生的稀疏解有很多的好处，如可以起到特征选择的作用，因为有些维的系数为0，说明这些维对于模型的作用很小。

这里有一个问题是，`L1` 正则化项不可微，所以无法像求 `L-BFGS` 那样去求。微软提出了`OWL-QN`(`Orthant-Wise Limited-Memory Quasi-Newton`)算法，该算法是基于`L-BFGS`算法的可用于求解L1正则的算法。 简单来讲，`OWL-QN`算法是指假定变量的象限确定的条件下使用`L-BFGS`算法来更新，同时，使得更新前后变量在同一个象限中(使用映射来满足条件)。

## C 对正则化的影响
【参考】
- [sklearn - L1 Penalty and Sparsity in Logistic Regression](https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic_l1_l2_sparsity.html#sphx-glr-auto-examples-linear-model-plot-logistic-l1-l2-sparsity-py)

在逻辑回归中使用正则化，不同的 C 值对稀疏解的影响：
![img](https://ws3.sinaimg.cn/large/69d4185bly1fx4b8mp88fj20hs0dcaa2.jpg)

可以看到 C 的值越大给予模型的自由度越大，即模型的非零解越多；C 值越小模型受到的约束越多，即会产生更多的非零解。从上图可以看到，在相同的 C 值下，`L1` 比 `L2` 会差生更多的非零解。